In [2]:
import causal_pyro
import pyro
import torch

from pyro.distributions import Normal, Uniform


In [ ]:
def ODE_example():
    # Hidden markov model with SIR ODE as latent
    
    beta  = pyro.sample("beta", Uniform(0, 1))
    gamma = pyro.sample("gamma", Uniform(0, 1)) 
    
    def SIR(dX, X):
        
        # TODO: Think about how to deal with name collisions below.
        SI_flux = pyro.deterministic("SI_flux", beta * X.S * X.I)
        IR_flux = pyro.deterministic("IR_flux", gamma * X.I)

        dX.S = -SI_flux
        dX.I = SI_flux - IR_flux
        dX.R = IR_flux

        # TODO: Think about how to deal with names below.
        pyro.sample("S_obs", Normal(X.S, 1.))
        return X.S

    init_state = {"S": 1.0, "I": 2.0, "R": 3.3, "t": 0.0}
    tspan = torch.tensor([1., 2., 3.])

    # Some representation of time, variable, and value.
    # This draft representation states that we observe `S_obs` at t=1.5 at value 3.7 and at t=2.5 at value 3.9.
    observed_data = {"S_obs": (torch.tensor([1.5, 2.5]), torch.tensor([3.7, 3.9]))}

    # This draft representation says that we intervene to set `SI_flux` at t=1.7 to value 0.0 
    # for the rest of the simulation.
    intervention_data = {"SI_flux": (torch.tensor([1.7]) torch.tensor([0.0]))}


    with ODE_solver():
        with condition(data=observed_data):
            with intervene(data=intervention_data, type="continuous"):
                S = simulate(SIR, init_state, tspan)